# 1(a) Importing BeautifulSoup and extracting the table from the wikipedia page

In [5]:
import requests
from bs4 import BeautifulSoup

wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

req = requests.get(wiki_url)
soup = BeautifulSoup(req.content, 'lxml')
table_classes = {"class": ["sortable", "plainrowheaders"]}
wikitables = soup.findAll("table", table_classes)

# 1(b) Extracting the table data and writing it into the file 'list_of_postal_codes_of_canada.tx

In [7]:
for table in wikitables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', "Neighbourhood"]:
        break
with open('list_of_postal_codes_of_canada.txt', 'w') as fo:
    for tr in soup.find('table').find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        PostalCode, Borough, Neighborhood = [td.text.strip() for td in tds[:3]]
        
        print('; '.join([PostalCode, Borough, Neighborhood]), file=fo)

# 1(c). Reading the initial CSV into a Pandas dataframe.

In [8]:

import pandas as pd

df = pd.read_csv('list_of_postal_codes_of_canada.txt', sep = ';', header = None, names = ['PostalCode', 'Borough', 'Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park)
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned


## 1(d). Stripping the rows of that do not have any 'Borough' in the dataframe.

In [10]:
import re
booleans = []

for result in df.Borough:
    if not re.search('Not assigned', result):
        booleans.append(True)
    else:
        booleans.append(False)
print(booleans[0:5])
print(len(booleans))

Filtered = pd.Series(booleans)

can_data = df[Filtered].reset_index(drop = True)

can_data.head(5)

[False, False, False, False, False]
20


,PostalCode,Borough,Neighborhood
0,M1HScarborough(Cedarbrae),M2HNorth York(Hillcrest Village),M3HNorth York(Bathurst Manor / Wilson Heights...
1,M1JScarborough(Scarborough Village),M2JNorth York(Fairview / Henry Farm / Oriole),M3JNorth York(Northwood Park / York University)
2,M1KScarborough(Kennedy Park / Ionview / East B...,M2KNorth York(Bayview Village),M3KNorth York(Downsview)East (CFB Toronto)
3,M1LScarborough(Golden Mile / Clairlea / Oakridge),M2LNorth York(York Mills / Silver Hills),M3LNorth York(Downsview)West
4,M1MScarborough(Cliffside / Cliffcrest / Scarbo...,M2MNorth York(Willowdale / Newtonbrook),M3MNorth York(Downsview)Central


## 1(e). Combining the rows based on similar PostalCode whilst aggregating the different neighborhoods separated with a comma.

In [12]:

new_cd = can_data.astype(str).groupby('PostalCode')['Neighborhood'].agg(','.join).reset_index()
merged_cd = pd.merge(can_data, new_cd, on = ['PostalCode'], how = 'inner')
duplicate_cd = merged_cd.drop(['Neighborhood_x'], axis = 1)
final_cd = duplicate_cd.drop_duplicates(subset='PostalCode')
final_cd.columns = ['PostalCode', 'Borough', 'Neighborhood']
final_cd = final_cd.reset_index(drop = True)

## 1(f). Replacing the rows in the 'Neighborhood' column that has a 'Borough' but lacks a 'Neighborhood' with NaN value.

In [13]:
import numpy as np
boolean = []

for result in final_cd.Neighborhood:
    if not re.search('Not assigned', result):
        boolean.append(True)
    else:
        boolean.append(False)



Filter = pd.Series(boolean)

X = final_cd['Neighborhood'].where(boolean, np.nan)
X = pd.Series(X)
final_cd = final_cd.drop(['Neighborhood'], axis = 1)

final_cd['Neighborhood'] = X.values

## 1(g). Replacing the Nan values in the 'Neighborhood' column with the value in the 'Borough'

In [15]:

final_cd["Neighborhood"] = final_cd["Neighborhood"].fillna(final_cd["Borough"])
final_cd.head()

,PostalCode,Borough,Neighborhood
0,M1HScarborough(Cedarbrae),M2HNorth York(Hillcrest Village),M3HNorth York(Bathurst Manor / Wilson Heights...
1,M1JScarborough(Scarborough Village),M2JNorth York(Fairview / Henry Farm / Oriole),M3JNorth York(Northwood Park / York University)
2,M1KScarborough(Kennedy Park / Ionview / East B...,M2KNorth York(Bayview Village),M3KNorth York(Downsview)East (CFB Toronto)
3,M1LScarborough(Golden Mile / Clairlea / Oakridge),M2LNorth York(York Mills / Silver Hills),M3LNorth York(Downsview)West
4,M1MScarborough(Cliffside / Cliffcrest / Scarbo...,M2MNorth York(Willowdale / Newtonbrook),M3MNorth York(Downsview)Central



## 1(h). The shape of the final version of the dataframe is being printed here.

In [16]:
final_cd.shape

(8, 3)

## 2(a). Importing Geographical Coordinates into the Dataframe

In [19]:
path = "http://cocl.us/Geospatial_data"

ll_df = pd.read_csv(path)
ll_df.columns = ['PostalCode', 'Latitude', 'Longitude']
ll_df.head()

HTTPError: HTTP Error 308: Permanent Redirect